## Empirical Evaluation of Sensitivity Bounds

We examine the sensitivity bounds provided by Best and Grauer, who provide bounds for:

* $|h_0|$: the norm of the optimal portfolio. 
* $|h_1|$: the norm of the deviation portfolio. 
* $|\mu_p - \mu_p^{*}|$: the difference between the mean returns of $h_0$ and $h_0 + h_1$, where $\mu_p = \mathbf{h}_0'\mathbf{\mu}$ and $\mu_p^{*} = \left(\mathbf{h}_0 + \mathbf{h}_1\right)'\left(\mathbf{\mu} + t\mathbf{q}\right)$
* $|\sigma_p^2 - \sigma_\hat{p}^2|$: the difference between variance of returns $\mathbf{h}_0$ and $\mathbf{h}_0 +\mathbf{h}_1$. The difference in variance between $h_0$ and $h_0+h_1$. 

We evaluate the bounds using parameters calculated from different datasets, representing realistic parameter choices of $(\Sigma, \mu)$. 

The function ```BGbounds()``` calculates the bounds, values, and actual quantities above. </br>
The function ```tangency2()``` calculates the tangency portfolio. </br>
The function ```Tol()``` is used to calculate the risk tolerance $T$ for the tangency portfolio. </br>
The function ```TestBounds()``` runs the simulation experiment to compare the bounds with the actual values from simulations.

To re-rerun experiments, run each cell in the notebook Results.ipynb

In [21]:
import numpy as np
import numpy.linalg as linalg
from numpy.linalg import eig, norm, cond
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import matplotlib
import seaborn as sns
import csv
import os

sns.set_style('darkgrid')

In [2]:
# tangency portfolio
def tangency2(Cov, mu, T):
    n = len(Cov)
    one = np.ones((n))
    invCov = np.linalg.inv(Cov)
    # Efficient set constants
    a = one.T@invCov@mu
    c = one.T@invCov@one
    # risk tolerance
#     T = 1/a
    # tangency portfolio
    w = (1/c)*(invCov@one) + T*(invCov@(mu - one*(a/c)))
    return w

In [3]:
# Best and Grauer Bounds
def BGbounds(Cov, mu, q):
    # Note that risk tolernance can be negative
    n = len(Cov)
    one = np.ones((n,1))
    invCov = np.linalg.inv(Cov)
    mu_hat = mu + q
    mu_hat = mu_hat/linalg.norm(mu_hat)*linalg.norm(mu) # fix norm of mu_hat = mu
    T = np.abs(1/(one.T@invCov@mu)) # risk tolerance corresponding to "true" tangency
    t = 1 # scale parameter for change in means vector
    # eigenvalues
    eig = np.linalg.eig(Cov)[0]
    l_max = np.max(eig)
    l_min = np.min(eig)
    norm_mu = np.linalg.norm(mu)
    norm_mu_hat = np.linalg.norm(mu_hat)
    norm_q = np.linalg.norm(q)
    # bounds
    BG_h0 = (1/np.sqrt(n))*(l_max/l_min) + T*(norm_mu_hat/l_min)*(1 + l_max/l_min)
    BG_h1 = t*T*(norm_q/l_min)*(1 + l_max/l_min)
    BG_mu = T*(norm_q/l_min)*(T*(1 + l_max/l_min)*(2*norm_mu + t*norm_q) + l_max/np.sqrt(n))
    BG_sigma = t*T*norm_q*(l_max/(l_min**2))*(1 + l_max/l_min) \
               *(T*(1 + l_max/l_min)*((2*norm_mu + t*norm_q) \
               *(1 + l_max/l_min)) + l_max/np.sqrt(n))
    
    return BG_h0, BG_h1, BG_mu, BG_sigma
    

In [4]:
# for debugging and deeper investigation
def BG_mean_bound(Cov, mu, q):
    n = len(Cov)
    one = np.ones((n,1))
    invCov = np.linalg.inv(Cov)
    mu_hat = mu + q
    mu_hat = mu_hat/linalg.norm(mu_hat)*linalg.norm(mu) # fix norm of mu_hat = mu
    T = np.abs(1/(one.T@invCov@mu)) # risk tolerance corresponding to "true" tangency
    t = 1 # scale parameter for change in means vector
    # eigenvalues
    eig = np.linalg.eig(Cov)[0]
    l_max = np.max(eig)
    l_min = np.min(eig)
    norm_mu = np.linalg.norm(mu)
    norm_mu_hat = np.linalg.norm(mu_hat)
    norm_q = np.linalg.norm(q)
    # bounds
    BG_mu = T*(norm_q/l_min)*(T*(1 + l_max/l_min)*(2*norm_mu + t*norm_q) + l_max/np.sqrt(n))
    term1 = T*(norm_q/l_min)
    term2 = T*(1 + l_max/l_min)
    term3 = (2*norm_mu + t*norm_q)
    term4 = l_max/np.sqrt(n)
    
    return np.array([T, term1, term2, term3, term4, BG_mu])

In [5]:
# for debugging and deeper investigation
def BG_cov_bound(Cov, mu, q):
    # Note that risk tolernance can be negative
    n = len(Cov)
    one = np.ones((n,1))
    invCov = np.linalg.inv(Cov)
    mu_hat = mu + q
    mu_hat = mu_hat/linalg.norm(mu_hat)*linalg.norm(mu) # fix norm of mu_hat = mu
    T = np.abs(1/(one.T@invCov@mu)) # risk tolerance corresponding to "true" tangency
    t = 1 # scale parameter for change in means vector
    # eigenvalues
    eig = np.linalg.eig(Cov)[0]
    l_max = np.max(eig)
    l_min = np.min(eig)
    norm_mu = np.linalg.norm(mu)
    norm_mu_hat = np.linalg.norm(mu_hat)
    norm_q = np.linalg.norm(q)
    # bounds
    BG_sigma = t*T*norm_q*(l_max/(l_min**2))*(1 + l_max/l_min) \
               *(T*(1 + l_max/l_min)*((2*norm_mu + t*norm_q) \
               *(1 + l_max/l_min)) + l_max/np.sqrt(n))
    term1 = t*T*norm_q*(l_max/(l_min**2))
    term2 = T*(1 + l_max/l_min)
    term3 = 2*norm_mu + t*norm_q
    term4 = l_max/np.sqrt(n)
    
    return np.array([T, term1, term2, term3, term4, BG_sigma])
    

In [23]:
# calculate risk tolerance for tangency portfolio
def Tol(cov, mu):
    n = len(cov)
    one = np.ones((n))
    invCov = np.linalg.inv(cov)
    # Efficient set constants
    a = one.T@invCov@mu
    return 1/a

In [7]:
def TestBounds(cov, mu, m):
    """
    Inputs
    ---------------------
    cov - covariance matrix
    mu - mean vector
    m - number of samples
    T - number of observations for sample. Determines variation in mu_hat. 
    """
   
    # initialise norm arrays
    norm_h0 = np.nan*np.zeros((m))
    norm_h1 = np.nan*np.zeros((m))
    h1_mu = np.nan*np.zeros((m))
    delta_sigma = np.nan*np.zeros((m))
     
    # Simulation Experiment
    #-----------------------
    # get dimension
    n = np.shape(mu)
    # risk tolerance
    T = 1/(np.ones((n)).T@linalg.inv(cov)@mu)
    # optimal tangency portfolio
    opt_t = tangency2(cov, mu, T)
    # condition number of covariance
    cond = np.linalg.cond(cov)
    
    # for each sample
    for i in range(0,m):
        # sample error vector from std MVN
        q = np.random.normal(0,1,n)
        # length of error vector 
        r = np.linalg.norm(mu)
        # normalise error vector
        q = q/np.linalg.norm(q)*r
        # calculate sample mean
        mu_hat = mu + q
        # fix norm
        mu_hat = mu_hat/linalg.norm(mu_hat)*linalg.norm(mu)
        # estimate tangency portfolio (assuming fixed covariance matrix)
        h0 = tangency2(cov, mu_hat, T)
        # calculate norm of tangency portfolio
        norm_h0[i] = np.linalg.norm(h0) 
        # calculate error portfolio
        h1 = opt_t - h0
        # calculate norm of error portfolio
        norm_h1[i] = np.linalg.norm(h1)
        # calculate difference in mean return
        h1_mu[i] = np.abs(h1.T@mu)
        # calculate change in variance
        delta_sigma[i] = np.abs(h1.T@cov@h0 + h1.T@cov@h1)
    
    # calculate bounds
    BG_h0, BG_h1, BG_mu, BG_sigma = BGbounds(cov, mu, q)
    
    # calculate ratio between norms and bounds
    diff_h0 = BG_h0/norm_h0
    diff_h1 = BG_h1/norm_h1
    diff_mu_p =  BG_mu/h1_mu
    diff_sigma_p = BG_sigma/delta_sigma    
    
    
    # Actual norms results
    Actual = pd.DataFrame(data = {'norm_h0': norm_h0, 'norm_h1': norm_h1, 
                                  'h1_mu': h1_mu, 'delta_sigma': delta_sigma})
    # BG Bounds
    Bounds = pd.DataFrame(data = {'norm_h0': BG_h0, 'norm_h1': BG_h1, 
                                  'h1_mu': BG_mu, 'delta_sigma': BG_sigma})
    
    # Differences
    Diff = pd.DataFrame(data = {'norm_h0': diff_h0, 'norm_h1': diff_h1, 
                                  'h1_mu': diff_mu_p, 'delta_sigma': diff_sigma_p})
    
    
    return Actual, Bounds, Diff



In [60]:
# Number of samples in all simulations
m = 100000 # samples

## Dataset 1: B&G data

In [25]:
# data from Best and Grauer (1991)

# Simulation parameters
bg_mu = np.array([1.01072,1.017618,1.018270,1.010761,1.019845,1.014452,1.009910,1.016353,1.013755,1.018315])
bg_mu -= 1
bg    = np.array([
        [0.251561e-2,0.765454e-3,0.110378e-2,0.131391e-2,0.157145e-3,0.554516e-3,0.936570e-3,0.164603e-2,0.509158e-3,0.151493e-2],
        [0.765454e-3,0.137432e-1,0.284739e-2,0.930502e-3,0.561023e-2,0.345666e-2,0.253434e-3,0.175684e-2,0.180949e-2,0.344478e-2],
        [0.110378e-2,0.284738e-2,0.139958e-1,0.102651e-2,0.424560e-2,0.276910e-2,0.758764e-3,0.319972e-2,0.327502e-2,0.362698e-2],
        [0.131391e-2,0.930502e-3,0.102651e-2,0.192771e-2,0.450576e-3,0.897736e-3,0.100989e-2,0.164109e-2,0.993300e-3,0.965801e-3],
        [0.157145e-3,0.561023e-2,0.424560e-2,0.450576e-3,0.159810e-1,0.349022e-2,0.713579e-3,0.421274e-2,0.297823e-2,0.439917e-2],
        [0.554516e-3,0.345666e-2,0.276910e-2,0.897736e-3,0.349022e-2,0.487226e-2,0.643392e-3,0.266937e-2,0.178289e-2,0.265065e-2],
        [0.936570e-3,0.253434e-3,0.758764e-3,0.100989e-2,0.713579e-3,0.643392e-3,0.166439e-2,0.101965e-2,0.635091e-3,0.611154e-3],
        [0.164603e-2,0.175684e-2,0.319972e-2,0.164109e-2,0.421274e-2,0.266937e-2,0.101965e-2,0.901327e-2,0.153437e-2,0.359597e-2],
        [0.509158e-3,0.180949e-2,0.327502e-2,0.993300e-3,0.297823e-2,0.178289e-2,0.635091e-3,0.153437e-2,0.573117e-2,0.215377e-2],
        [0.151493e-2,0.344478e-2,0.362698e-2,0.965801e-3,0.439917e-2,0.265065e-2,0.611154e-3,0.359597e-2,0.215377e-2,0.140409e-1]
        ])

# condition number
bg_cond = np.linalg.cond(bg)
# bounds vs actual
norm_bg1, bounds_bg1, diff_bg1 =  TestBounds(bg, bg_mu, m)

print(norm_bg1.min(), '\n\n', norm_bg1.median(), '\n\n', norm_bg1.max())
bounds_bg1

## Dataset 2: J&K

In [48]:
# data from Jobson and Korkie (1980)

# get data
jk_mu  = np.array(pd.read_excel('data/JK_data.xlsx', sheet_name='return')).flatten()
jk_cov  = pd.read_excel('data/JK_data.xlsx', sheet_name='cov')
jk_cov  = np.array(jk_cov)
jk_cond =  np.linalg.cond(jk_cov)

# Simulation Parameters
# mean and covariance
jk_mu = jk_mu/100
jk_cov = jk_cov/(100**2)
n = len(cov)

# condition number
cond = np.linalg.cond(cov)

# calculate bounds vs actuals
norm_jk, bounds_jk, diff_jk =  TestBounds(jk_cov, jk_mu, m)
print(norm_jk.min(), '\n\n', norm_jk.median(), '\n\n', norm_jk.max())

norm_h0        7.348858e-01
norm_h1        7.069418e-01
h1_mu          1.304969e-07
delta_sigma    1.543860e-08
dtype: float64 

 norm_h0        1.876321
norm_h1        1.806790
h1_mu          0.002086
delta_sigma    0.000247
dtype: float64 

 norm_h0        4.169358
norm_h1        4.090660
h1_mu          0.012220
delta_sigma    0.001446
dtype: float64


## Dataset 3: JSE data

In [27]:
# load JSE data
jse = pd.read_excel('data/jse_returns_data.xlsx', sheet_name='nolog')
jse = jse.drop(['Dates'], axis=1)

# calculate moments
jse_mu = np.array(jse.mean())
jse_cov = np.array(jse.cov())
jse_cond = np.linalg.cond(jse_cov)

norm_jse, bounds_jse, diff_jse =  TestBounds(jse_cov, jse_mu, m)


In [14]:
print(norm_jse.min(), '\n\n', norm_jse.median(), '\n\n', norm_jse.max())

norm_h0        7.198402e-01
norm_h1        7.072584e-01
h1_mu          3.087001e-08
delta_sigma    3.733383e-10
dtype: float64 

 norm_h0        38.090094
norm_h1        38.127235
h1_mu           0.003441
delta_sigma     0.000042
dtype: float64 

 norm_h0        200.555733
norm_h1        199.985652
h1_mu            0.020002
delta_sigma      0.000242
dtype: float64


## Dataset 4-11: FF Industry Portfolios

In [61]:
# diff_df

with open('table_results1.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerow(['Data set', 'Condition number', 'min', 'median', 'max', 'min', 'median', 'max', 'min', 'median', 'max', 'min', 'median','max'])
    writer.writerow(['BG', bg_cond, diff_bg1.min()[0], diff_bg1.mean()[0], diff_bg1.max()[0], 
                     diff_bg1.min()[1], diff_bg1.mean()[1], diff_bg1.max()[1], 
                     diff_bg1.min()[2], diff_bg1.mean()[2], diff_bg1.max()[2], 
                     diff_bg1.min()[3], diff_bg1.mean()[3], diff_bg1.max()[3]])
    writer.writerow(['JK', jk_cond, diff_jk.min()[0], diff_jk.mean()[0], diff_jk.max()[0], 
                     diff_jk.min()[1], diff_jk.mean()[1], diff_jk.max()[1], 
                     diff_jk.min()[2], diff_jk.mean()[2], diff_jk.max()[2],
                     diff_jk.min()[3], diff_jk.mean()[3], diff_jk.max()[3]])
    writer.writerow(['JSE', jse_cond, diff_jse.min()[0], diff_jse.mean()[0], diff_jse.max()[0], 
                     diff_jse.min()[1], diff_jse.mean()[1], diff_jse.max()[1], 
                     diff_jse.min()[2], diff_jse.mean()[2], diff_jse.max()[2], 
                     diff_jse.min()[3], diff_jse.mean()[3], diff_jse.max()[3]])
    added = []
    for file_name in sorted(os.listdir('data/ff_data')):
        
        if file_name.endswith('.CSV'):
            # import data
            df = pd.read_csv( os.path.join('data/ff_data',file_name), header = 6, engine = 'python',  nrows = 1121, index_col =0 )
            # replace missing values
            df = df.replace(-99.99, np.NaN)
            # rescale returns
            df /= 100
            dfname = 'Ind'+file_name.split('_',1)[0]
            added.append(dfname)
            exec("%s=df"%dfname)
            print("writing file %s to file"%file_name)

            # calculate moments
            df_mu = np.array(df.mean())
            df_cov = np.array(df.cov())
            df_cond = np.linalg.cond(df_cov)
            # calculate bounds
            norm_df, bounds_df, diff_df =  TestBounds(df_cov, df_mu, m)
            
            # write to csv            
            writer.writerow([dfname, df_cond, diff_df.min()[0], diff_df.mean()[0], diff_df.max()[0], diff_df.min()[1], diff_df.mean()[1], diff_df.max()[1], diff_df.min()[2], diff_df.mean()[2], diff_df.max()[2], diff_df.min()[3], diff_df.mean()[3], diff_df.max()[3]])
        
print("Data added: "+", ".join(added))



writing file 10_Industry_Portfolios.CSV to file
writing file 12_Industry_Portfolios.CSV to file
writing file 17_Industry_Portfolios.CSV to file
writing file 30_Industry_Portfolios.CSV to file
writing file 38_Industry_Portfolios.CSV to file
writing file 48_Industry_Portfolios.CSV to file
writing file 49_Industry_Portfolios.CSV to file
writing file 5_Industry_Portfolios.CSV to file
Data added: Ind10, Ind12, Ind17, Ind30, Ind38, Ind48, Ind49, Ind5


In [62]:
# norm_df

with open('table_results_norm1.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerow(['Data set', 'Condition number', 'min', 'median', 'max', 'min', 'median', 'max', 'min', 'median', 'max', 'min', 'median','max'])
    writer.writerow(['BG', bg_cond, norm_bg1.min()[0], norm_bg1.mean()[0], norm_bg1.max()[0],
                     norm_bg1.min()[1], norm_bg1.mean()[1], norm_bg1.max()[1],
                     norm_bg1.min()[2], norm_bg1.mean()[2], norm_bg1.max()[2]
                     norm_bg1.min()[3], norm_bg1.mean()[3], norm_bg1.max()[3]])
    writer.writerow(['JK', jk_cond, norm_jk.min()[0], norm_jk.mean()[0], norm_jk.max()[0],
                     norm_jk.min()[1], norm_jk.mean()[1], norm_jk.max()[1], 
                     norm_jk.min()[2], norm_jk.mean()[2], norm_jk.max()[2], 
                     norm_jk.min()[3], norm_jk.mean()[3], norm_jk.max()[3]])
    writer.writerow(['JSE', jse_cond, norm_jse.min()[0], norm_jse.mean()[0], norm_jse.max()[0],
                     norm_jse.min()[1], norm_jse.mean()[1], norm_jse.max()[1],
                     norm_jse.min()[2], norm_jse.mean()[2], norm_jse.max()[2],
                     norm_jse.min()[3], norm_jse.mean()[3], norm_jse.max()[3]])

    # FF datasets
    added = []
    for file_name in sorted(os.listdir('ff_data')):
        if file_name.endswith('.CSV'):
            df = pd.read_csv( os.path.join('ff_data',file_name), header = 6, engine = 'python',  nrows = 1121, index_col =0 )
            # replace missing data with NaNs
            df = df.replace(-99.99, np.NaN)
            # rescale from percentage to units
            df /= 100
            dfname = 'Ind'+file_name.split('_',1)[0]
            added.append(dfname)
            exec("%s=df"%dfname)
            print("writing file %s to file"%file_name)

            # calculate moments
            df_mu = np.array(df.mean())
            df_cov = np.array(df.cov())
            df_cond = np.linalg.cond(df_cov)
            # compare bounds vs actual
            norm_df, bounds_df, diff_df =  TestBounds(df_cov, df_mu, m)

            # write to csv            
            writer.writerow([dfname, df_cond, norm_df.min()[0], norm_df.mean()[0], norm_df.max()[0], norm_df.min()[1], norm_df.mean()[1], norm_df.max()[1], norm_df.min()[2], norm_df.mean()[2], norm_df.max()[2], norm_df.min()[3], norm_df.mean()[3], norm_df.max()[3]])
            
            
# check datasets added
print("Data added: "+", ".join(added))

writing file 10_Industry_Portfolios.CSV to file
writing file 12_Industry_Portfolios.CSV to file
writing file 17_Industry_Portfolios.CSV to file
writing file 30_Industry_Portfolios.CSV to file
writing file 38_Industry_Portfolios.CSV to file
writing file 48_Industry_Portfolios.CSV to file
writing file 49_Industry_Portfolios.CSV to file
writing file 5_Industry_Portfolios.CSV to file
Data added: Ind10, Ind12, Ind17, Ind30, Ind38, Ind48, Ind49, Ind5


In [71]:
# bounds_df

with open('table_results_bounds1.csv', 'w') as file:    
    df_bounds = pd.DataFrame(columns = ['Data set', '1/sqrt(n)', 'cond', 'inv(l_min)', 'mu', 't', 'T', 'h0', 'h1', 'delta_mu', 'delta_sigma' ])
    df_bounds.loc[0] = ['BG', 1/np.sqrt(len(bg_mu)), bg_cond, 1/eig(bg)[0].min(), norm(bg_mu), norm(bg_mu)/2, Tol(bg,bg_mu), 
                        bounds_bg1.norm_h0[0], bounds_bg1.norm_h1[0], bounds_bg1.h1_mu[0], bounds_bg1.delta_sigma[0]]
    df_bounds.loc[1] = ['JK', 1/np.sqrt(len(jk_mu)), jk_cond, 1/eig(jk_cov)[0].min(), norm(jk_mu), norm(jk_mu)/2, Tol(jk_cov,jk_mu), 
                        bounds_jk.norm_h0[0], bounds_jk.norm_h1[0], bounds_jk.h1_mu[0], bounds_jk.delta_sigma[0]]
    df_bounds.loc[2] = ['JSE', 1/np.sqrt(len(jse_mu)), jse_cond, 1/eig(jse_cov)[0].min(), norm(jse_mu), norm(jse_mu)/2, Tol(jse_cov,jse_mu), 
                        bounds_jse.norm_h0[0], bounds_jse.norm_h1[0], bounds_jse.h1_mu[0], bounds_jse.delta_sigma[0]]
    
    added = []
    i = 3 # counter
    for file_name in sorted(os.listdir('data/ff_data')):
        
        if file_name.endswith('.CSV'):
            df = pd.read_csv( os.path.join('data/ff_data',file_name), header = 6, engine = 'python',  nrows = 1121, index_col =0 )
            # replace missing with NaNs
            df = df.replace(-99.99, np.NaN)
            # rescalt percentages to units
            df /= 100
            dfname = 'Ind'+file_name.split('_',1)[0]
            added.append(dfname)
            exec("%s=df"%dfname)
            print("writing file %s to file"%file_name)
   
            # calculate moments
            df_mu = np.array(df.mean())
            df_cov = np.array(df.cov())
            df_cond = np.linalg.cond(df_cov)
            # store moments for each FF dataset
            exec("%s_mu = np.array(df.mean())"%dfname)
            exec("%s_cov = np.array(df.cov())"%dfname)
            exec("%s_cond = np.linalg.cond(df_cov)"%dfname)
            # compare bounds to actuals
            norm_df, bounds_df, diff_df =  TestBounds(df_cov, df_mu, m)
            exec("norm_%s, bounds_%s, diff_%s =  TestBounds(df_cov, df_mu, m)"%(dfname, dfname, dfname))
            
            # wtite to DF
            df_bounds.loc[i] = [dfname, 1/np.sqrt(len(df_mu)), df_cond, 1/eig(df_cov)[0].min(), norm(df_mu), norm(df_mu/2), Tol(df_cov, df_mu), 
                                bounds_df.norm_h0[0], bounds_df.norm_h1[0], bounds_df.h1_mu[0], bounds_df.delta_sigma[0]]
            # increase counter
            i+=1 
            
        
print("Data added: "+", ".join(added))

# change index of Ind5 results
df_bounds.loc[2.5] = df_bounds.loc[10]
df_bounds = df_bounds.sort_index().drop(10).reset_index(drop=True)
# write to csv
df_bounds.to_csv('table_results_bounds1.csv')
df_bounds

writing file 10_Industry_Portfolios.CSV to file
writing file 12_Industry_Portfolios.CSV to file
writing file 17_Industry_Portfolios.CSV to file
writing file 30_Industry_Portfolios.CSV to file
writing file 38_Industry_Portfolios.CSV to file
writing file 48_Industry_Portfolios.CSV to file
writing file 49_Industry_Portfolios.CSV to file
writing file 5_Industry_Portfolios.CSV to file
Data added: Ind10, Ind12, Ind17, Ind30, Ind38, Ind48, Ind49, Ind5


,Data set,1/sqrt(n),cond,inv(l_min),mu,t,T,h0,h1,delta_mu,delta_sigma
0,BG,0.316228,47.816169,1478.124266,0.048661,0.024330,0.109005,3.978560e+02,3.827352e+02,6.170571,6.941651e+05
1,JK,0.223607,47.099865,1005.671668,5.140457,2.570229,0.001183,3.047093e+02,2.941775e+02,5.431153,5.849990e+05
2,JSE,0.288675,5534.039408,477973.135449,0.044249,0.022125,0.012094,1.417372e+06,1.415775e+06,2273.781653,3.853622e+14
3,Ind5,0.447214,37.976407,2812.578234,0.022076,0.011038,0.221445,5.528929e+02,5.359093e+02,7.942572,4.535578e+05
4,Ind10,0.316228,122.610158,4444.619678,0.031471,0.015736,0.154587,2.711622e+03,2.672850e+03,39.199130,7.308585e+07
5,Ind12,0.288675,151.977117,4345.875126,0.034573,0.017287,0.150264,3.497685e+03,3.453813e+03,54.056783,1.914513e+08
6,Ind17,0.242536,197.456472,3682.459330,0.040658,0.020329,0.146538,4.402031e+03,4.354141e+03,78.111026,6.052474e+08
7,Ind30,0.182574,291.431715,3004.442004,0.055630,0.027815,0.132628,6.535553e+03,6.482345e+03,143.874527,3.575914e+09
8,Ind38,0.164399,387.913607,3333.307987,0.063092,0.031546,0.111524,9.185350e+03,9.121577e+03,192.993846,1.129735e+10
9,Ind48,0.144338,461.227974,2812.365693,0.072704,0.036352,0.114123,1.085257e+04,1.078600e+04,269.033708,2.645722e+10


# Additional Calcs

1. Comparing largest bound (JSE) to smallest bound (Ind5) for $|\mathbf{h}_0|$ and $|\mathbf{h}_1|$

In [57]:
bounds_jse/bounds_Ind5

,norm_h0,norm_h1,h1_mu,delta_sigma
0,2563.556974,2641.817974,286.277762,8.496431e+08


In [80]:
(norm_jse.max())/(norm_Ind5.max())

norm_h0        14.186533
norm_h1        14.204922
h1_mu           2.507731
delta_sigma     0.136956
dtype: float64